# 1) Preparação e leitura dos dados

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

#importando os datasets:
# teste
dftest = pd.read_csv('./datafinalteste.csv')
xtes = dftest.iloc[:,3:54]
y_test = dftest.iloc[:, 55]
# Padronização das caracteristicas de x. 
ss = StandardScaler()
x_test = pd.DataFrame(ss.fit_transform(xtes), columns=xtes.columns)

#Tratar os dados da coluna label
#y_test=pd.get_dummies (ytes, drop_first=False)
#y_test



In [ ]:
# treino
dftrain = pd.read_csv('./datafinaltreino.csv')
xtre = dftrain.iloc[:,3:54]
y_train = dftrain.iloc[:, 55]

# Padronização das caracteristicas de x. 
ss = StandardScaler()
x_train = pd.DataFrame(ss.fit_transform(xtre), columns=xtre.columns)

#Tratar os dados da coluna label
#y_train=pd.get_dummies (ytre, drop_first=False)
#y_train


In [ ]:
# validação
dfvalidate = pd.read_csv('./datafinalvalida.csv')
xval = dfvalidate.iloc[:,3:54]
y_val = dfvalidate.iloc[:, 55]

# Padronização das caracteristicas de x. 
ss = StandardScaler()
x_val = pd.DataFrame(ss.fit_transform(xval), columns=xval.columns)

#Tratar os dados da coluna label
#y_validation=pd.get_dummies (yval, drop_first=False)
#y_val

# 2) Método com XGBoost - (utilizando bases de treino e teste)

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.model_selection import train_test_split

# Dividindo data set em treinamento e teste (random_state = seed):
#x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 30)

# Configurando o xgboost na base de treino
import xgboost
xgboost = xgboost.XGBClassifier()
xgboost.fit(x_train, y_train)

# Configurando a predição base de teste
y_pred = xgboost.predict(x_test)

# Resultado 1
print(metrics.accuracy_score(y_test,y_pred))

# Resultado 2 
print (metrics.classification_report(y_test,y_pred))


# 3) Método com XGBoost e GridSearch - (utilizando bases de treino e teste)

In [ ]:
import xgboost
from sklearn.model_selection import GridSearchCV

xgboost = xgboost.XGBClassifier()

parameters = {'nthread':[4], 
              'objective':['binary:logistic'],
              'learning_rate': [0.05], 
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], 
              'missing':[-999],}

grid= GridSearchCV(xgboost, param_grid=parameters)
grid_result = grid.fit(x_train, y_train)

#resultados
print("Melhor: %f Usando %s" % (grid_result.best_score_, grid_result.best_params_))



# 3.1 - Como utilizar os resultados obtidos em (3) nas bases de validação e teste? e Verificar acurácia final do modelo? Aqui fiz direto sem utilizar a base de validação. 

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# Aqui treinei o modelo com os melhores parametros obtidos em (3)
import xgboost
xgboost = xgboost.XGBClassifier(colsample_bytree = 0.7, learning_rate = 0.05, max_depth = 6, min_child_weight = 11, missing = -999, n_estimators = 5, nthread = 4, objective = 'binary:logistic', seed = 1337, silent = 1, subsample = 0.8)
xgboost.fit(x_train, y_train)

# Configurando a predição base de teste
y_pred1 = xgboost.predict(x_test)

# Resultado 1
print(metrics.accuracy_score(y_test,y_pred1))

# Resultado 2 
print (metrics.classification_report(y_test,y_pred1))

# 4) Exemplo 1 com Logistic Regression e GridSearch (1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

dual=[True,False]
max_iter=[100,110,120,130,140]
param_grid = dict(dual=dual,max_iter=max_iter)

lr = LogisticRegression(penalty='l2')
grid1 = GridSearchCV(estimator=lr, param_grid=param_grid, n_jobs=-1)

grid_result1 = grid1.fit(x_train, y_train)
print("Melhor: %f Usando %s" % (grid_result1.best_score_, grid_result1.best_params_))



# 4.1 - Como utilizar os resultados obtidos em (4) nas bases de validação e teste? e Verificar acurácia final do modelo?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

dual=[False]
max_iter=[100]
param_grid = dict(dual=dual,max_iter=max_iter)

lr = LogisticRegression(penalty='l2')
gridx = GridSearchCV(estimator=lr, param_grid=param_grid, n_jobs=-1)

grid_resultx = gridx.fit(x_train, y_train)

# Configurando a predição base de teste
y_predx = xgboost.predict(x_test)

# Resultado 1
print(metrics.accuracy_score(y_test,y_predx))

# Resultado 2 
print (metrics.classification_report(y_test,y_predx))




# 5) Exemplo 2 com Logistic Regression e GridSearch 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

dual=[True,False]
max_iter=[100,110,120,130,140]
C = [1.0,1.5,2.0,2.5]
param_grid = dict(dual=dual,max_iter=max_iter,C=C)

lr = LogisticRegression(penalty='l2')
grid2 = GridSearchCV(estimator=lr, param_grid=param_grid, n_jobs=-1)

# Base de treinamento
grid_result2 = grid2.fit(x_train, y_train)
print("Melhor: %f Usando %s" % (grid_result2.best_score_, grid_result2.best_params_))


# 5.1 - Como utilizar os resultados obtidos em (5) nas bases de validação e teste? e Verificar acurácia final do modelo?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

ols = LinearRegression()
ols_params = {'fit_intercept':[True, False],
              'normalize':[True, False]}
grid3 = GridSearchCV(ols, param_grid=ols_params, scoring=metricas, verbose=100, refit='neg_mean_squared_error', return_train_score=False)

grid_result3 = grid3.fit(x_train, y_train)

print("Melhor: %f Usando %s" % (grid_result3.best_score_, grid_result3.best_params_))
